## Project Template (Deepnote)

You can use this notebook as a starting point for your class project and/or the extra exercise for graduate students in Deepnote. It comes set up with the dependencies we use in class, so you don't have to install these yourself.

**Notes:**
- To use this in your own workspace, click “Duplicate” for the project in Deepnote.
- Class dependencies, namely `drake` and the `manipulation` package, are already preinstalled in the Deepnote image for this course, as well as a bunch of other dependencies that these packages depend on or that we use in class
- To see the full list of installed dependencies, as well as version numbers etc, see this file: [pyproject.toml](https://github.com/RussTedrake/manipulation/blob/master/pyproject.toml).
  - **Note**: we install all the extra dependencies, including the `dev` dependencies, into the Docker image that this deepnote project runs on.


In [1]:
%load_ext autoreload
%autoreload 2
# ^ make it so we can change inner files


In [2]:

import numpy as np
from pydrake.all import (
    ContactModel,
    DiagramBuilder,
    MeshcatVisualizer,
    MultibodyPlant,
    Rgba,
    RigidTransform,
    RollPitchYaw,
    SceneGraph,
    Simulator,
    Sphere,
    StartMeshcat,
    VectorLogSink,
    VisualizationConfig,
)

from manipulation.station import (
    AddPointClouds,
    MakeHardwareStation,
    Parser,
)
from scripts.load_scenario import load_scenario

In [3]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7000


Click the link above to open Meshcat in your browser!


In [4]:
use_hydroelastic = True # this takes forever

scenario = load_scenario()

# since makehardwarestation finalizes the plant before giving it back to us
def callback(parser: Parser):
    plant = parser.plant()
    plant.set_contact_model(ContactModel.kHydroelasticWithFallback)
    scene_graph: SceneGraph = parser.scene_graph()
    config = scene_graph.get_config()
    if use_hydroelastic:
        # one object rigid, all else compliant? table
        # HE modulus parameters
        # allow broom to turn more easily

        # add "handle" to broom to make it easier to grip?
        config.default_proximity_properties.compliance_type = "compliant"
    scene_graph.set_config(config)

station = MakeHardwareStation(scenario, meshcat=meshcat, parser_prefinalize_callback=callback)
plant: MultibodyPlant = station.plant()


builder = DiagramBuilder()
builder.AddSystem(station)

to_point_cloud = AddPointClouds(
    scenario=scenario, station=station, builder=builder, meshcat=meshcat
)
builder.ExportOutput(
    to_point_cloud["camera0"].point_cloud_output_port(), "point_cloud0"
)

# diagram = builder.Build()


OutputPortIndex(0)

In [ ]:
from scripts.grasp_broom import plan_path, grasp_path, build_temp_plant

# basic test
# X_WG1 = RigidTransform(RollPitchYaw(np.pi/2, 0, 0), [0, 0.5, 1])
# X_WG2 = RigidTransform(RollPitchYaw(np.pi/2, 0, np.pi/2), [0, 1, 1])

# meshcat.SetObject("start", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
# meshcat.SetTransform("start", X_WG1)
# meshcat.SetObject("goal", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
# meshcat.SetTransform("goal", X_WG2)


ImportError: cannot import name 'combine_trajectory' from 'scripts.grasp_broom' (/workspaces/my_project/src/6.4210/sweep-bot/scripts/grasp_broom.py)

In [ ]:
# test function to see the temp plant

# diagram, plant, gripper_frame = build_temp_plant(q0 = None, meshcat = meshcat)
# config = VisualizationConfig(lcm_bus='default', publish_period=0.015625, publish_illustration=True, default_illustration_color=Rgba(r=0.9, g=0.9, b=0.9, a=1.0), publish_proximity=True, default_proximity_color=Rgba(r=0.8, g=0.0, b=0.0, a=1.0), initial_proximity_alpha=0.5, publish_contacts=True, publish_inertia=True, enable_meshcat_creation=True, delete_on_initialization_event=True, enable_alpha_sliders=False)
# context = diagram.CreateDefaultContext()
# diagram.ForcedPublish(context)
# meshcat.SetObject("gripper", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
# meshcat.SetTransform("gripper", gripper_frame.GetPoseInWorld(context))


In [ ]:
from pydrake.systems.primitives import TrajectorySource, ConstantVectorSource
from scripts.ik import solve_ik_for_pose

# builder, plant, station = make_builder()

# grasp broom
X_WStart = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0, 1.0, 0.5])
X_WPregrasp = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0.6, 1.7, 0.5])
X_WGrasp = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0.6, 1.60, 0.5])
X_WLift = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0.6, 1.60, 0.7])

for pos in [X_WPregrasp, X_WGrasp, X_WLift]:
    print(solve_ik_for_pose(plant, pos))
# AddMeshcatTriad(meshcat, 'a', X_PT=X_WStart)
# AddMeshcatTriad(meshcat, 'b', X_PT=X_WGoal)


traj, wsg_traj = grasp_path(X_WStart, X_WPregrasp, X_WGrasp, X_WLift)

# print 20 joint positions sampled
times = np.linspace(traj.start_time(), traj.end_time(), 20)
for t in times:
    print(f'time{t}: {traj.value(t).flatten()}')

In [ ]:
# # POSITION CONTROL
# q_src = builder.AddSystem(TrajectorySource(traj))
# builder.Connect(q_src.get_output_port(), station.GetInputPort("iiwa.position"))
# wsg_src = builder.AddSystem(TrajectorySource(wsg_traj))
# builder.Connect(wsg_src.get_output_port(), station.GetInputPort("wsg.position"))

# diagram = builder.Build()


# ID CONTROLLER
_, iiwa_plant, _ = build_temp_plant()
ID_controller = builder.AddSystem(
    InverseDynamicsController(
        iiwa_plant,
        kp = np.ones(7) * 200,
        kd = np.ones(7) * 40,
        ki = np.zeros(7),
        has_reference_acceleration=False,
    )
)


control_estimated_state_input_port = ID_controller.get_input_port(0)
control_desired_state_input_port = ID_controller.get_input_port(1)
control_output_port = ID_controller.get_output_port(0)


state_src = builder.AddSystem(TrajectorySource(traj, output_derivative_order=1))

builder.Connect(
    state_src.get_output_port(),
    control_desired_state_input_port
)

builder.Connect(
    station.GetOutputPort("iiwa_state"),
    control_estimated_state_input_port
)

builder.Connect(
    control_output_port,
    station.GetInputPort("iiwa_actuation")
)

wsg_src = builder.AddSystem(TrajectorySource(wsg_traj))
builder.Connect(wsg_src.get_output_port(), station.GetInputPort("wsg.position"))

diagram = builder.Build()

In [ ]:
from scripts.broom_utils import get_broom_pregrip, get_broom_grip,compute_broom_grasp_angle
from scripts.meta_controller import pregrip_to_grip_trajectory

plant_context = plant.CreateDefaultContext()
base = plant.GetBodyByName("base", plant.GetModelInstanceByName('iiwa'))
body = plant.GetBodyByName("handle_link")
X_WBroom = plant.EvalBodyPoseInWorld(plant_context, body)
X_WBase = plant.EvalBodyPoseInWorld(plant_context, base)
ang = compute_broom_grasp_angle(X_WBroom, X_WBase.translation())
ang = np.pi
# X_WPregrip = get_broom_pregrip(X_WBroom, ang)
X_WPregrip = X_WGoal
X_WGrip = get_broom_grip(X_WBroom, ang)
AddMeshcatTriad(meshcat, 'broom', X_PT=X_WBroom)
AddMeshcatTriad(meshcat, 'base', X_PT=X_WBase)
AddMeshcatTriad(meshcat, 'pregrip', X_PT=X_WPregrip)
AddMeshcatTriad(meshcat, 'grip', X_PT=X_WGrip)

builder, plant, station = make_builder()

traj_V_G, traj_wsg = plan_path(X_WStart, X_WPregrip)
traj = traj_V_G
traj2_V_G, traj2_wsg_command = pregrip_to_grip_trajectory(X_WPregrip, X_WGrip)

# traj_V_G = combine_trajectory(traj1_V_G, traj2_V_G)
# traj_wsg = combine_trajectory(traj1_wsg, traj2_wsg)

# grip the broom

# print 20 joint positions sampled
times = np.linspace(traj.start_time(), traj.end_time(), 20)
for t in times:
    print(f'time{t}: {traj.value(t).flatten()}')

# add traj to builder and connect
q_src = builder.AddSystem(TrajectorySource(traj))
builder.Connect(q_src.get_output_port(), station.GetInputPort("iiwa.position"))
wsg_src = builder.AddSystem(TrajectorySource(traj_wsg))
builder.Connect(wsg_src.get_output_port(), station.GetInputPort("wsg.position"))

diagram = builder.Build()
X_WPregrip


In [ ]:

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
meshcat.StartRecording()
simulator.Initialize()
simulator.AdvanceTo(traj.end_time())
meshcat.StopRecording()
meshcat.PublishRecording()